[복습]

 1. Selenium 크롬 브라우저 사용했었다.
 
 2. 화면에 안 뜨게 할 수도 있고, GUI 아닌 브라우저도 있음. 

 3. 데이터

     - 텍스트가 아닌 것들은 모두 binary라고 보자.
     - 대표적으로 이미지, 영상 등. 
 
 

 4. 로그인 데이터
 
     - 셀레늄만 가능한 건 아니다.
     - 로그인도 request와 response이기 때문에, request 시 받는 인증 토큰을 네트워크 반응 분석해서 요청 보내주면 된다.

# 이미지 크롤링

 웹 상의 이미지도 결국은 데이터. 
 
## 이미지 저장 방법

1. 개념적 저장 : 해당 이미지의 URL 저장 
    - 실제 파일 저장 X
    - URL 저장
    
    
2. 물리적 저장 : 이미지 실제 파일을 저장. 
    - 실제 이미지를 내 컴퓨터에 저장하고,
    - 저장된 이미지의 경로(파일 경로)를 저장한다.

> 1번 방법의 경우, 서버에서 삭제되거나 rename될 경우 접근되지 않는 문제가 생긴다. xbox처럼 이미지가 도출되지 않을 수도 있다.
> 따라서 파일 입출력을 사용한다.
> 쓰기 모드로 파일을 열고, binary 모드로 쓴 후 파일을 닫는다.

### 개념적 저장 예시

In [1]:
import requests

In [14]:
url = "https://search.pstatic.net/common?type=o&size=150x112&quality=95&direct=true&src=http%3A%2F%2Fsstatic.naver.net%2Fkeypage%2Fimage%2Fdss%2F146%2F48%2F40%2F78%2F146_2484078_team_image_url_1484014683897.jpg"
req = requests.get(url)
req

<Response [200]>

In [18]:
# 이미지 데이터 텍스트로 확인하면 binary 형태로 보임.
req.content[:100]

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02'

### 물리적 저장 예시

 위에서 보낸 request를 바탕으로 저장한다.

In [19]:
img_name = 'kia_tigers_logo'

# 파일 열고, 바이너리 모드로 쓰기.
with open(f"./{img_name}", 'wb') as f:
    f.write(req.content)

# ----------------- 20200605 수업 -----------------

In [35]:
# module import
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import parse_qs, urlparse

**os 모듈** : 운영체제 관리.

# 네이버 뉴스 크롤링(with image)

> 네이버 뉴스를 이미지 경로, description 함께 저장하자.

In [20]:
# url 설정
base_url = "https://search.naver.com/search.naver"
params = {
    'where': 'news',
    'query': '환율전쟁'
}

# 요청 전송
req = requests.get(base_url, params)
req

<Response [200]>

In [28]:
# 뉴스 관련 정보와 이미지 url을 저장하자.

soup = BeautifulSoup(req.text)

news_list = soup.select('ul.type01 > li')

# 제목, 이미지 소스, url 저장
news_results = []
for news in news_list:
    a_tag = news.find('a', class_='_sp_each_title')
    
    title = a_tag.get_text(strip=True)
    url = a_tag.get('href')
        
    news_dict = {
        'title': title,
        'url': url
    }
    
    image_wrap = news.find('div', class_='thumb')    
    if image_wrap: # 이미지 있으면
        news_dict['img'] = image_wrap.find('img').get('src')
    
    news_results.append(news_dict)

In [29]:
news_results

[{'title': '미·중 갈등 확산…환율전쟁조짐까지',
  'url': 'http://news.kbs.co.kr/news/view.do?ncd=4455353&ref=A',
  'img': 'https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F056%2F2020%2F05%2F26%2F10841598.jpg&type=ofullfill80_80_q75_re2'},
 {'title': '코로나발 ‘쩐의 전쟁’…환율전쟁더 복잡하게 전개된다',
  'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=050&aid=0000053885',
  'img': 'https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F050%2F2020%2F06%2F03%2F53885.jpg&type=ofullfill80_80_q75_re2'},
 {'title': "미·중환율전쟁에 홍콩보안법까지…산업계 '전전긍긍'",
  'url': 'http://yna.kr/AKR20200527090300003?did=1195m',
  'img': 'https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F001%2F2020%2F05%2F27%2F11637257.jpg&type=ofullfill80_80_q75_re2'},
 {'title': '中, 이틀 연속 위안화 절하…환율‘힘겨루기’ 본격화',
  'url': 'https://cnbc.sbs.co.kr/article_hub/10000984244?division=NAVER',
  'img': 'https://search.pstatic.net/comm

 위에서 이미지 url을 저장했다.
 
 **url 저장**

- 비용은 많이 들지 않는다.
- 데이터 일관성이 떨어지고, 바뀔 수 있다.

**binary 저장**
 
 아래와 같이 코드를 수정해 보자.
     

### 1) binary 데이터 저장

 바이너리 데이터로 이미지를 저장한다. 이 상태로 DB에 저장하기도 한다. 바이트 데이터 언제든 뽑아 와서 렌더링할 수 있다.

In [30]:
for result in news_results:
    image_url = result['img']
    
    if image_url: # 이미지 url이 있으면,
        req = requests.get(image_url)
        result['img_content'] = req.content

In [33]:
news_results[0] # 이미지가 바이너리 데이터로 저장된다.

{'title': '미·중 갈등 확산…환율전쟁조짐까지',
 'url': 'http://news.kbs.co.kr/news/view.do?ncd=4455353&ref=A',
 'img': 'https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F056%2F2020%2F05%2F26%2F10841598.jpg&type=ofullfill80_80_q75_re2',
 'img_content': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x00P\x00P\x03\x01!\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1b\x00\x01\x00\x03\x00\x03\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x04\x05\x01\x02\x06\x07\xff\xc4\x009\x10\x00\x02\x01\x03\x02\x05\x02\x03\x04\x07\t\x00\x00\x00\x00\x00\x01\x02\x03\x00\x04\x11\x05!\x06\x12\x131AQa\x14\x81\x91\x15"q\xc1\x07\x1634B\xa1\xb2$\x82\x92\x93

### 2) 파일 디렉토리 생성해서 저장

- 파일 이름 지정할 때, `parse_qs` 이용해서 쿼리셋 기반으로 저장한다.
- 이미지 저장을 위한 로컬 용량 소모된다.
- 그러나 서버 상에서 사라지거나 편집되는 등 걱정하지 않아도 된다.(= 더 안정적이다.)

![img-saved](./images/web_30.png)

In [44]:
# 베이스 파일 경로
base_file_dir = 'images_saved'

# 파일 경로 검사 및 생성
if not os.path.exists(base_file_dir):
    os.makedirs(base_file_dir)

# 저장한 이미지 경로에 요청을 보내고, 로컬 파일 디렉토리에 저장.
for result in news_results:
    image_url = result['img']

    
    if image_url: # 저장된 이미지 경로가 있으면,
        
        # 파일 이름을 만들기 위해 쿼리셋 생성.
        parsed_image_url = urlparse(image_url) 
        parsed_image_queries = parse_qs(parsed_image_url.query)
        image_src = parsed_image_queries['src'][0]
        
        # 파일 경로와 이름을 통해 이미지 저장 경로 생성.
        image_path = os.path.join(base_file_dir, image_src.split('/')[-1])
        print(image_path)
        
        # 이미지 가져온 후 저장.
        image_content = requests.get(image_url).content
        with open(image_path, 'wb') as f:
            f.write(image_content)
        
        #이미지 경로도 저장
        result['img_path'] = image_path

# 확인
news_results[0]

images_saved\10841598.jpg
images_saved\53885.jpg
images_saved\11637257.jpg
images_saved\212366.jpg
images_saved\3288113.jpg
images_saved\1547007.jpg
images_saved\541005.jpg
images_saved\3742561.jpg
images_saved\1547017.jpg
images_saved\11641183.jpg


{'title': '미·중 갈등 확산…환율전쟁조짐까지',
 'url': 'http://news.kbs.co.kr/news/view.do?ncd=4455353&ref=A',
 'img': 'https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F056%2F2020%2F05%2F26%2F10841598.jpg&type=ofullfill80_80_q75_re2',
 'img_content': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x00P\x00P\x03\x01!\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1b\x00\x01\x00\x03\x00\x03\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x04\x05\x01\x02\x06\x07\xff\xc4\x009\x10\x00\x02\x01\x03\x02\x05\x02\x03\x04\x07\t\x00\x00\x00\x00\x00\x01\x02\x03\x00\x04\x11\x05!\x06\x12\x131AQa\x14\x81\x91\x15"q\xc1\x07\x1634B\xa1\xb2$\x82\x92\x93

**큰 데이터**

이미지의 경우 메모리가 크지 않아서 한 번에 메모리에 올릴 수 있다. 그러나 유튜브 동영상 등 한 번에 메모리에 올리기 부담스러운 파일들의 경우, 다른 방식을 사용한다.

 위와 같은 코드에, 큰 데이터라고 가정해 보겠다.
 
* 한 번에 오지 않는다. stream 옵션을 준다. response가 여러 번 오기 때문에.
* 잘려서 오는 데이터를 chunk 단위로 저장한다. 

In [45]:
# 베이스 파일 경로
base_file_dir = 'BIG-images_saved'

if not os.path.exists(base_file_dir):
    os.makedirs(base_file_dir)

# 큰 데이터 가정
for result in news_results:
    image_url = result['img']

    if image_url: 
        
        parsed_image_url = urlparse(image_url) 
        parsed_image_queries = parse_qs(parsed_image_url.query)
        image_src = parsed_image_queries['src'][0]        
        image_path = os.path.join(base_file_dir, image_src.split('/')[-1])
        
        # 큰 데이터 : 한 번에 오지 않는다.
        image_content = requests.get(image_url, stream=True).content # stream 옵션
                
        # chunk 단위로 저장
        with open(image_path, 'wb') as f:
            for chunk in req.iter_content():
                f.write(chunk)
                
        #이미지 경로도 저장
        result['BIG-img_path'] = image_path

# 확인
news_results[0]

{'title': '미·중 갈등 확산…환율전쟁조짐까지',
 'url': 'http://news.kbs.co.kr/news/view.do?ncd=4455353&ref=A',
 'img': 'https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F056%2F2020%2F05%2F26%2F10841598.jpg&type=ofullfill80_80_q75_re2',
 'img_content': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x00P\x00P\x03\x01!\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1b\x00\x01\x00\x03\x00\x03\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x04\x05\x01\x02\x06\x07\xff\xc4\x009\x10\x00\x02\x01\x03\x02\x05\x02\x03\x04\x07\t\x00\x00\x00\x00\x00\x01\x02\x03\x00\x04\x11\x05!\x06\x12\x131AQa\x14\x81\x91\x15"q\xc1\x07\x1634B\xa1\xb2$\x82\x92\x93